## BiLSRM-CRF implementation from Pytorch Tutorials

In [2]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim

torch.manual_seed(114514)

In [5]:
def prepare_sequence(seq, word2idx):
    idxs = [word2idx[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

def log_sum_exp(vec):
    max_score = vec[0, torch.argmax(vec, dim=1)] # avoid overflow
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
           torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

In [7]:
class BiLISM_CRF(nn.Module):
    
    def __init__(self, vocab_size, tag2idx, embedding_dim, hidden_dim):
        super(BiLSTM_CRF, self).__init__()
        assert hidden_sim % 2 == 0, 'hidden-dim must be even for BiLSTM'
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag2idx = tag2idx
        self.tagset_size = len(tag2idx)
        
        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=1, bidirectional=True)
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)
        # transition scores matrix A
        self.transitions = nn.Parameter(torch.randn(self.tagset_size, self.tagset_size))
        self.transitions.data[tag2idx[START_TAG], :] = -1000
        self.transitions.data[:, tag2idx[STOP_TAG]] = -1000
        
        # "a hidden layer between c_i and the CRF layer"
        self.hidden = (torch.randn(2, 1, self.hidden_dim // 2),
                       torch.randn(2, 1, self.hidden_dim // 2))
        
    def _forward_alg(self, feats):
        # forward algorithm (DP through time): compute denominator Z for loss
        init_alphas = torch.full((1, self.tagset_size), -10000.)
        init_alphas[0][self.tag2idx[START_TAG]] = 0.
        
        forward_var = init_alphas
        for feat in feats:
            alphas_t = []
            for next_tag in range(self.tagset_size): # parallel?
                emit_score = feat[next_tag].view(1, -1).expand(1, self.tagset_size)
                trans_score = self.transitions[next_tag].view(1, -1)
                next_tag_var = forward_var + trans_score + emit_score
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag2idx[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha
        
    def _score_sentence(self, feats, tags):
        # compute score of tag sequence, ie. numerator for loss
        score = torch.zeros(1)
        tags = torch.cat([torch.LongTensor([self.tag2idx[START_TAG]]), tags])
        for i, feat in enumerate(feats):
            score += self.transitions[tags[i], tags[i + 1]] + feat[tags[i+1]]
        score += self.transitions[self.tag2idx[STOP_TAG], tag[-1]]
        return score
        
        
    def _viterbi_decode(self, feats):
        # compute best path and its score for test
        backpointers = []
        init_vvars = torch.full((1, self.tagset_size), -10000.)
        init_vvars[0][self.tag2idx[START_TAG]] = 0
        
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []
            viterbivars_t = []
            
            for next_tag in range(self.tagset_size): # ||
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = torch.argmax(next_tag_var, dim=1)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)
        
        # transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag2idx[STOP_TAG]]
        best_tag_id = torch.argmax(terminal_var, dim=1)
        path_score = terminal_var[0][best_tag_id]
        
        # follow the back pointers to decode best path
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        
        # pop off the START_TAG
        start = best_path.pop()
        assert start == self.tag2idx[START_TAG] # Sanity check
        best_path.reverse()
        return path_score, best_path
    
    def _get_lstm_features(self, sentence):
        self.hidden = self.
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats
    
    def neg_log_likelihood(self, sentence, tags):
        feats = self._get_lstm_features(sentence)
        forward_score = self._forward_alg(feats) # Z
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score
        
    def forward(self, sentence): # for test?
        lstm_feats = self._get_lstm_features(sentence)
        score, tag_seq = self._viterbi_decode(lstm_feats)
        return score, tag_seq

SyntaxError: invalid syntax (<ipython-input-7-b8ec82184ed3>, line 91)

In [ ]:
START_TAG = "<START>"
STOP_TAG = "<STOP>"

training_data = [(
    "the wall street journal reported today that apple corporation made money".split(),
    "B I I I O O O B I O O".split()
), (
    "georgia tech is a university in georgia".split(),
    "B I O O O O B".split()
)]

word2idx = {}
for sentence, tags in training_data:
    for word in sentence:
        if word not in word2idx:
            word2idx[word] = len(word2idx)

tag_to_ix = {"B": 0, "I": 1, "O": 2, START_TAG: 3, STOP_TAG: 4}

modle = BiLSTM_CRF(len(word2idx), tag2idx, embedding_dim=5, hidden_dim=4)
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-4)

with torch.no_grad():
    precheck_sent = prepare_sequence(traning_data[0][0], word2idx)
    precheck_tags = torch.tensor([tag2idx[t] for t in training_data[0][1]], 
                                dtype=torch.long)
    print(model(precheck_sent))


for epoch in range(300):
    for sentence, tags in training_data:
        model.zero_grad()
        sentence_in = prepare_sequence(sentence, word2idx)
        targets = torch.tensor([tag2idx[t] for t in tags], dtype=torch.long)
        loss = model.neg_log_likelihood(sentence_in, targets)
        loss.backward()
        optimizer.step()

with torch.no_grad():
    precheck_sent = prepare_senquence(training_data[0][0], word2idx)
    print(model(precheck_sent))